In [3]:
!pip install torchsummary

In [284]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [286]:
# 데이터셋 인스턴스 생성

df = pd.read_csv('C:/AI/6week/BP_data.csv')  # CSV 파일 경로
df.drop(columns=["Patient_Number"], inplace=True)
df.fillna(df.mean(numeric_only=True), inplace=True)

In [288]:
df

,Blood_Pressure_Abnormality,Level_of_Hemoglobin,Genetic_Pedigree_Coefficient,Age,BMI,Sex,Pregnancy,Smoking,Physical_activity,salt_content_in_the_diet,alcohol_consumption_per_day,Level_of_Stress,Chronic_kidney_disease,Adrenal_and_thyroid_disorders
0,1,11.28,0.90,34,23,1,1.000000,0,45961,48071,251.008532,2,1,1
1,0,9.75,0.23,54,33,1,0.450226,0,26106,25333,205.000000,3,0,0
2,1,10.79,0.91,70,49,0,0.450226,0,9995,29465,67.000000,2,1,0
3,0,11.00,0.43,71,50,0,0.450226,0,10635,7439,242.000000,1,1,0
4,1,14.17,0.83,52,19,0,0.450226,0,15619,49644,397.000000,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,10.14,0.02,69,26,1,0.450226,1,26118,47568,144.000000,3,1,0
1996,1,11.77,1.00,24,45,1,1.000000,1,2572,8063,251.008532,3,1,1
1997,1,16.91,0.22,18,42,0,0.450226,0,14933,24753,251.008532,2,1,1
1998,0,11.15,0.72,46,45,1,0.450226,1,18157,15275,253.000000,3,0,1


In [290]:
# 데이터와 타겟 분리
X = df.drop(columns=["Blood_Pressure_Abnormality"])
y = df["Blood_Pressure_Abnormality"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [292]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [294]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [296]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32)

In [298]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1600, 13), (400, 13), (1600,), (400,))

# 모델 정의

In [301]:
class BPModel(nn.Module):
    def __init__(self, input_dim):
        super(BPModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 1)  # 4 classes in the dataset
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
input_dim = X_train_tensor.shape[1]  # 또는 X_train.shape[1]
model = BPModel(input_dim)

# 이진 분류

In [304]:
pos_weight = torch.tensor([float((y_train == 0).sum() / (y_train == 1).sum())])
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습

In [307]:
for epoch in range(20):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        predicted = (torch.sigmoid(outputs) >= 0.5).int()
        correct += (predicted == labels.int()).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    accuracy = correct / total * 100
    print(f"Epoch {epoch+1}/20, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("Training complete.")

Epoch 1/20, Loss: 0.6700, Accuracy: 65.12%
Epoch 2/20, Loss: 0.6281, Accuracy: 71.69%
Epoch 3/20, Loss: 0.5904, Accuracy: 72.81%
Epoch 4/20, Loss: 0.5595, Accuracy: 72.94%
Epoch 5/20, Loss: 0.5387, Accuracy: 73.62%
Epoch 6/20, Loss: 0.5244, Accuracy: 74.12%
Epoch 7/20, Loss: 0.5134, Accuracy: 74.62%
Epoch 8/20, Loss: 0.5038, Accuracy: 74.88%
Epoch 9/20, Loss: 0.4950, Accuracy: 75.88%
Epoch 10/20, Loss: 0.4859, Accuracy: 76.62%
Epoch 11/20, Loss: 0.4763, Accuracy: 77.75%
Epoch 12/20, Loss: 0.4666, Accuracy: 78.31%
Epoch 13/20, Loss: 0.4571, Accuracy: 79.56%
Epoch 14/20, Loss: 0.4471, Accuracy: 80.06%
Epoch 15/20, Loss: 0.4365, Accuracy: 81.19%
Epoch 16/20, Loss: 0.4266, Accuracy: 81.69%
Epoch 17/20, Loss: 0.4162, Accuracy: 82.88%
Epoch 18/20, Loss: 0.4059, Accuracy: 83.69%
Epoch 19/20, Loss: 0.3965, Accuracy: 84.25%
Epoch 20/20, Loss: 0.3863, Accuracy: 85.19%
Training complete.


In [309]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Test the model
model.eval()  # 평가 모드로 전환
y_true = []
y_pred = []

with torch.no_grad():  # 그래디언트 계산을 하지 않음
    for inputs, labels in test_loader:
        outputs = model(inputs).squeeze()  # 모델 예측
        predicted = (torch.sigmoid(outputs) >= 0.5).int()  # 0.5를 기준으로 이진 분류
        y_true.extend(labels.numpy())  # 실제 값 저장
        y_pred.extend(predicted.numpy())  # 예측값 저장

# 평가 지표 계산
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

# 결과 출력
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")


Test Accuracy: 0.8225
Test Precision: 0.8182
Test Recall: 0.8223
Test F1-Score: 0.8203
Test ROC-AUC: 0.8225
Confusion Matrix:
[[167  36]
 [ 35 162]]
